In [19]:
import numpy as np
import astropy.units as u
import os
import re

In [20]:
masses_in_Emass = [12]
fenvs = [0.1]
cwd = os.getcwd()
# cwd_linstyle = re.sub("C:\\","\\/mnt\/c\/",cwd)
# cwd_linstyle = re.sub("\\\\","\/",cwd_linstyle)
print("masses in Emass = ", masses_in_Emass)
print("fenvs = ", fenvs)
# print("cwd_linstyle = ", cwd_linstyle)
# print("cwd = ", cwd)

masses in Emass =  [12]
fenvs =  [0.1]


In [21]:
# print certain lines of the file that we use as base to edit and create files for different models
with open('inlist_2a_evolve') as f:
    lines = f.readlines()
    print(lines[7], lines[10], lines[29], lines[53])
    f.close

      saved_model_name = "planet_1d_setS_15ME_0.1_9.0.mod"
       save_model_filename = "planet_2a_evolve_15ME_0.1_9.0.mod"
       pause_before_terminate = .true. ! lets you examine the final pgstar window when run terminates
 	star_history_name = "history_2a_evolve_15ME_0.1_9.0"



In [22]:
# create a list of arrays with all the lines of the files we need to write
# and a list of filenames to save the files with
files_to_write = []
fnames = []
for j,fenv in enumerate(fenvs):
    for i,mass in enumerate(masses_in_Emass):
        with open('inlist_2a_evolve') as f:
            lines = f.readlines()
            #saved_model_name = "planet_1d_setS_15ME_0.1_9.0.mod"
            lines[7] = "\tsaved_model_name = \"planet_1d_setS_"+str(mass)+"ME_"+str(fenv)+"_9.0.mod\"\n"
            lines[10] = "\tsave_model_filename = \"planet_2a_evolve_"+str(mass)+"ME_"+str(fenv)+"_9.0.mod\"\n"
            lines[29] = "\tpause_before_terminate = .false. ! lets you examine the final pgstar window when run terminates"
            lines[53] = "\tstar_history_name = \"history_planet_2a_evolve_"+str(mass)+"ME_"+str(fenv)+"_9.0\"\n"
            files_to_write.append(lines)
            fnames.append("inlist_2a_evolve_"+str(mass)+"ME_"+str(fenv)+"_9.0")
            f.close
        print(fnames[-1])
        print("\n")

inlist_2a_evolve_12ME_0.1_9.0




In [23]:
# check if the lists have been updated and saved correctly
print(files_to_write[-1][7],files_to_write[-1][10],files_to_write[-1][29],files_to_write[-1][53])

	saved_model_name = "planet_1d_setS_12ME_0.1_9.0.mod"
 	save_model_filename = "planet_2a_evolve_12ME_0.1_9.0.mod"
 	pause_before_terminate = .false. ! lets you examine the final pgstar window when run terminates 	star_history_name = "history_planet_2a_evolve_12ME_0.1_9.0"



In [24]:
# write out to files
for i,w_files in enumerate(fnames):
    with open(str(w_files),'w') as f:
        f.writelines(files_to_write[i])
        f.close

In [25]:
# make execution script
'''steps:
    - add the #!/bin line that makes the file executable
    - loop over the filenames that we need to run and write script for each in the following steps
        1. rename to inlist
        2. clean
        3. make
        4. run
        5. rename to previous name
        6. create a copy of log folder with unique name
        7. display the contents of the directory
'''

script = ["#!/bin/sh\n\n"]
for fn in fnames:
    script.append("mv "+fn+" inlist\n")
    script.append("./clean\n")
    script.append("./mk\n")
    script.append("./rn\n")
    script.append("mv "+"inlist "+fn+"\n")
    script.append("mv LOGS LOGS_"+fn+"\n")
    script.append("ls\n")

In [26]:
# if working remotely, this command helps copy the right files to remote from local,
# might have to edit the directories to suit yourself

namestring = "script_2a "
for n in fnames:
    namestring = namestring + n + " "

print("scp "+namestring+"qadeer@student62.strw.leidenuniv.nl:/data2/qadeer/eia2022/eia_lab2/")

scp script_2a inlist_2a_evolve_12ME_0.1_9.0 qadeer@student62.strw.leidenuniv.nl:/data2/qadeer/eia2022/eia_lab2/


In [27]:
# write out to the bash script to a txt file
with open('script_2a.sh','w') as f:
    f.writelines(script)
    f.close

# Loop of inlist update sequence from Lab1 for reference

In [11]:
# print certain lines of the file that we use as base to edit and create files for different models
with open('inlist_1d_setS') as f:
    lines = f.readlines()
    print(lines[7], lines[10], lines[52])
    f.close

      saved_model_name = "planet_1c_reducemass_15ME_0.1.mod"
       save_model_filename = "planet_1d_setS_15ME_0.1_9.0.mod"
 	star_history_name = "history_1d_setS_15ME_0.1_9.0"



In [12]:
# create a list of arrays with all the lines of the files we need to write
# and a list of filenames to save the files with
files_to_write = []
fnames = []
for j,fenv in enumerate(fenvs):
    for i,mass in enumerate(masses_in_Emass):
        with open('inlist_1d_setS') as f:
            lines = f.readlines()
            lines[7] = "\tsaved_model_name = \"planet_1c_reducemass_"+str(m_multiples[i])+"ME_"+str(fenv)+".mod\"\n"
            lines[10] = "\tsave_model_filename = \"planet_1d_setS_"+str(m_multiples[i])+"ME_"+str(fenv)+"_9.0.mod\"\n"
            lines[52] = "\tstar_history_name = \"history_1d_setS_"+str(m_multiples[i])+"ME_"+str(fenv)+"_9.0\"\n"
            files_to_write.append(lines)
            fnames.append("inlist_1d_setS_"+str(m_multiples[i])+"ME_"+str(fenv)+"_9.0")
            f.close
        print(fnames[-1])
        print("\n")

inlist_1d_setS_3ME_0.1_9.0


inlist_1d_setS_5ME_0.1_9.0


inlist_1d_setS_7ME_0.1_9.0


inlist_1d_setS_10ME_0.1_9.0


inlist_1d_setS_12ME_0.1_9.0


inlist_1d_setS_3ME_0.01_9.0


inlist_1d_setS_5ME_0.01_9.0


inlist_1d_setS_7ME_0.01_9.0


inlist_1d_setS_10ME_0.01_9.0


inlist_1d_setS_12ME_0.01_9.0




In [17]:
# check if the lists have been updated and saved correctly
print(files_to_write[-1][7],files_to_write[-1][10],files_to_write[-1][52])

	saved_model_name = "planet_1c_reducemass_12ME_0.01.mod"
 	save_model_filename = "planet_1d_setS_12ME_0.01_9.0.mod"
 	star_history_name = "history_1d_setS_12ME_0.01_9.0"



In [13]:
# write out to files
for i,w_files in enumerate(fnames):
    with open(str(w_files),'w') as f:
        f.writelines(files_to_write[i])
        f.close

In [14]:
# make execution script
'''steps:
    - add the #!/bin line that makes the file executable
    - loop over the filenames that we need to run and write script for each in the following steps
        1. rename to inlist
        2. clean
        3. make
        4. run
        5. rename to previous name
        6. create a copy of log folder with unique name
        7. display the contents of the directory
'''

script = ["#!/bin/sh\n\n"]
for fn in fnames:
    script.append("mv "+fn+" inlist\n")
    script.append("./clean\n")
    script.append("./mk\n")
    script.append("./rn\n")
    script.append("mv "+"inlist "+fn+"\n")
    script.append("mv LOGS LOGS_"+fn+"\n")
    script.append("ls\n")

In [15]:
# if working remotely, this command helps copy the right files to remote from local,
# might have to edit the directories to suit yourself

namestring = "script_1d "
for n in fnames:
    namestring = namestring + n + " "

print("scp "+namestring+"qadeer@student33.strw.leidenuniv.nl:/home/qadeer/Desktop/eia2022/ass1/Lab1/Lab1/")

scp script_1d inlist_1d_setS_3ME_0.1_9.0 inlist_1d_setS_5ME_0.1_9.0 inlist_1d_setS_7ME_0.1_9.0 inlist_1d_setS_10ME_0.1_9.0 inlist_1d_setS_12ME_0.1_9.0 inlist_1d_setS_3ME_0.01_9.0 inlist_1d_setS_5ME_0.01_9.0 inlist_1d_setS_7ME_0.01_9.0 inlist_1d_setS_10ME_0.01_9.0 inlist_1d_setS_12ME_0.01_9.0 qadeer@student33.strw.leidenuniv.nl:/home/qadeer/Desktop/eia2022/ass1/Lab1/Lab1/


In [16]:
# write out to the bash script to a txt file
with open('script_1d.txt','w') as f:
    f.writelines(script)
    f.close